In [2]:
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64
import os
from bson.json_util import dumps

# Importing AnimalShelter class
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "test"
password = "test"
animals = AnimalShelter(username, password)

# Class read method must support return of cursor object
df = pd.DataFrame.from_records(animals.read({}))

#########################
# Dashboard Layout / View
#########################

# Create dash application
app = Dash(__name__)

# Image handling
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image path
if not os.path.exists(image_filename):
    raise FileNotFoundError(f"Image file '{image_filename}' not found.")

encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div(id='main-div', children=[
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('CS-340'))),
    html.Center(html.B(html.H1('Amber Law Project 2 Dashboard'))),
    html.Hr(),
    html.Div(
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image'),
        style={
            'display': 'flex',
            'justifyContent': 'center',
            'alignItems': 'center',
            'height': '100vh',  # Full viewport height
            'margin': '0'
        }
    ),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        ),
        html.Br(),
        dcc.Input(id='search-box', type='text', placeholder='Search for animals...', debounce=True),
    ]),
    html.Button('Select All Rows', id='select-all-btn', n_clicks=0),
    html.Button('Download Data as CSV', id='download-btn', n_clicks=0),
    dcc.Download(id='download-component'),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    dcc.RadioItems(
        id='chart-selector',
        options=[
            {'label': 'Histogram', 'value': 'hist'},
            {'label': 'Pie Chart', 'value': 'pie'},
            {'label': 'Bar Chart', 'value': 'bar'},
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Box Plot', 'value': 'box'},
            {'label': 'Heatmap', 'value': 'heatmap'}
        ],
        value='hist',
        inline=True
    ),
    html.Hr(),
    html.Button('Toggle Dark/Light Mode', id='theme-toggle-btn', n_clicks=0),
    html.Hr(),
    html.Div(id='theme-div', className='row',
             style={'display': 'block'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Hr(),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Select all rows callback
@app.callback(
    Output('datatable-id', 'selected_rows'),
    Input('select-all-btn', 'n_clicks'),
    State('datatable-id', 'data')
)
def select_all_rows(n_clicks, data):
    return list(range(len(data))) if n_clicks % 2 == 1 else []

# Download data callback
@app.callback(
    Output('download-component', 'data'),
    Input('download-btn', 'n_clicks'),
    State('datatable-id', 'data')
)
def download_data(n_clicks, data):
    if n_clicks:
        df = pd.DataFrame(data)
        return dcc.send_data_frame(df.to_csv, 'filtered_data.csv')

# Theme toggle callback
@app.callback(
    Output('theme-div', 'style'),
    Input('theme-toggle-btn', 'n_clicks')
)
def toggle_theme(n_clicks):
    return {'backgroundColor': '#2c2c2c', 'color': '#f5f5f5', 'display': 'block'} if n_clicks % 2 == 1 else {'display': 'block'}
    
# Radio button filters
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value'),
     Input('search-box', 'value')]
)
def update_dashboard(filter_type, search_value):
    if filter_type == 'water':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'mount':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(animals.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }))
    else:
        df = pd.DataFrame.from_records(animals.read({}))

    if search_value:
        df = df[df.apply(lambda row: row.astype(str).str.contains(search_value, case=False).any(), axis=1)]
        
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart callback
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('chart-selector', 'value')]
)
def update_graphs(viewData, chart_type):
    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        return html.Div("No data available for the chart.")

    if chart_type == 'hist':
        fig = px.histogram(
            dff,
            x='age_upon_outcome_in_weeks',
            nbins=20,
            title='Distribution of Animals by Age (in Weeks)'
        )
        description = "This histogram shows the distribution of animals by their age in weeks."

    elif chart_type == 'pie':
        fig = px.pie(
            dff,
            names='sex_upon_outcome',
            title='Outcome Distribution by Sex'
        )
        description = "This pie chart shows the proportion of different outcomes by the sex of the animals."

    elif chart_type == 'bar':
        fig = px.bar(
            dff,
            x='breed',
            title='Count of Animals by Breed'
        )
        description = "This bar chart shows the count of animals for each breed."

    elif chart_type == 'scatter':
        fig = px.scatter(
            dff,
            x='age_upon_outcome_in_weeks',
            y='outcome_type',
            title='Age vs Outcome Type'
        )
        description = "This scatter plot shows the relationship between the age of animals (in weeks) and their outcome type."

    elif chart_type == 'line':
        # Ensure 'age_upon_outcome_in_weeks' column has numeric values
        dff['age_upon_outcome_in_weeks'] = pd.to_numeric(dff['age_upon_outcome_in_weeks'], errors='coerce')
        dff = dff.dropna(subset=['age_upon_outcome_in_weeks'])
    
        # Sort by age and calculate cumulative count
        age_counts = dff['age_upon_outcome_in_weeks'].value_counts().sort_index()
        cumulative_counts = age_counts.cumsum()
    
        fig = px.line(
            x=cumulative_counts.index,
            y=cumulative_counts.values,
            title='Cumulative Number of Animals by Age (in Weeks)',
            labels={'x': 'Age in Weeks', 'y': 'Cumulative Number of Animals'}
        )
        description = "This line chart shows the cumulative number of animals by their age in weeks."
    elif chart_type == 'box':
        # Box Plot: Age distribution by outcome type
        fig = px.box(
            dff,
            x='outcome_type',
            y='age_upon_outcome_in_weeks',
            title='Age Distribution by Outcome Type'
        )
        description = "This box plot shows the age distribution of animals for each outcome type."

    elif chart_type == 'heatmap':
        # Heatmap: Correlation matrix for numerical columns
        numeric_columns = dff.select_dtypes(include=['number'])
        if not numeric_columns.empty:
            correlation_matrix = numeric_columns.corr()
            fig = px.imshow(
                correlation_matrix,
                labels=dict(x="Columns", y="Columns", color="Correlation"),
                title='Heatmap of Correlation Matrix'
            )
            description = "This heatmap shows the correlation matrix for numerical columns in the dataset."
        else:
            return html.Div("No numerical columns available for a heatmap.")
            
    return html.Div([
        html.Div(description, style={'margin-bottom': '10px', 'font-weight': 'bold'}),
        dcc.Graph(figure=fig)
    ])
    
# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty or len(row_ids) == 0:
        return html.Div("No data available for the map select some to show if you already didn't.")

    # Check if the required latitude and longitude columns exist
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return html.Div("Latitude and Longitude columns are missing.")

    # Collect all markers for the selected rows
    markers = []
    for idx in row_ids:
        try:
            lat = float(dff.loc[idx, 'location_lat'])
            lon = float(dff.loc[idx, 'location_long'])
            breed = dff.loc[idx, 'breed']
            name = dff.loc[idx, 'name']
            
            markers.append(
                dl.Marker(position=[lat, lon],
                          children=[
                              dl.Tooltip(breed),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(name)
                              ])
                          ])
            )
        except (ValueError, KeyError, TypeError):
            continue  # Skip rows with invalid latitude/longitude

    # If no valid markers were created, display a message
    if not markers:
        return html.Div("No valid latitude and longitude data available for the selected rows.")

    # Return the map with all markers
    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[float(dff.loc[row_ids[0], 'location_lat']), float(dff.loc[row_ids[0], 'location_long'])],
               zoom=10,
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   *markers  # Unpack all markers into the map
               ])
    ]


# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', port=8050, debug=True)